In [6]:
'''
Manual rebasing script

Purpose: this script takes unrebased projection output and manually calculates the 'combined, rebased impact'
(correctly!) by enacting this equation:

combined_rebased_impact = minlost_hi_rebased * riskshare_hi + minlost_lo_rebased * (1 - riskshare_hi)

This is done for histclim and fulladapt scenarios. As per usual, the histclim value will then be subtracted
from the fulladapt value and we will get our final result.

Parameters:

@model     : the name of the model (and thus the name of folder that output is stored in)
@scenario  : the name of the scenario from which projection output is sourced

Outputs: a modified version of the original projection file that now includes a new column, 'rebased_new',
which has this correctly-calculated combined, rebased impact.

'''
__author__ = 'Kit Schwarz'
__contact__ = 'csschwarz@uchicago.edu'
__version__ = '1.0'

############
# LIBRARIES
############

import xarray as xr
import pandas as pd
import getpass
import sys

In [7]:
############
# PARAMETERS
############

model = 'special_single'
# select: uninteracted_main_model_w_chn, uninteracted_main_model

############
# PATHWAYS
############

username = getpass.getuser()

if model == 'uninteracted_main_model_w_chn':
    
    proj_root = ('/shares/gcp/outputs/labor/impacts-woodwork/uninteracted_main_model_w_chn/' +
                 'uninteracted_splines_w_chn_21_37_41_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3')
    
elif model == 'uninteracted_main_model':
    
    proj_root = ('/shares/gcp/outputs/labor/impacts-woodwork/test_rcc/' +
                 'uninteracted_splines_27_37_39_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3')

else:

    sys.exit("Your model is unrecognized.")
    
############
# FUNCTION
############

def rebase_df(proj_root, model, scenario):
    dt = (xr.open_dataset(
            f'{proj_root}/{model}{scenario}.nc4')
            .to_dataframe()
            .reset_index()
         )

    # to get rid of duplicate rows (because of orderofoperations column)
    dt = dt.loc[dt.orderofoperations == 'clip'] 

    # get a sub-dataset for the base years used to calculate rebasing
    base = dt.loc[(dt.year >=2001) & (dt.year <=2010)]

    # get mean by region for the base years
    mean_base = base.groupby("regions").agg(
                        high_base=('highriskimpacts', 'mean'),
                         low_base=('lowriskimpacts', 'mean'))

    # merge base year values onto main dataset so we can calculate new columns
    dt = dt.merge(mean_base,
                left_on='regions',
                right_index = True)

    # these are the rebased sector-specific impacts
    dt['rebased_high'] = dt.highriskimpacts - dt.high_base
    dt['rebased_low'] = dt.lowriskimpacts - dt.low_base

    # and the final weighted average (combined, rebased) impacts
    # this is how manual_rebasing_MC.py and manual_rebasing_single.py calculate rebased_new
    dt['rebased_new'] = (dt.rebased_high * dt['clip']) + (dt.rebased_low * (1 - dt['clip']))
    
    return dt

In [41]:
########################
# IMPLEMENT MANUAL CHECK
########################

fulladapt = rebase_df(proj_root, model, '')
histclim = rebase_df(proj_root, model, '-histclim')


# these are rebased total impacts
fulladapt['impact_H'] = (
            (fulladapt['highriskimpacts'] - fulladapt['high_base'])
            - (histclim['highriskimpacts'] - histclim['high_base'])
            )

fulladapt['impact_L'] = (
            (fulladapt['lowriskimpacts'] - fulladapt['low_base'])
            - (histclim['lowriskimpacts'] - histclim['low_base'])
            )

# these are the calculation of terms found in Ashwin's memo on the 
# projection system mess-up
fulladapt['weighted_term'] = (
                (fulladapt['clip'] * fulladapt['impact_H'])
                + ((1- fulladapt['clip']) * fulladapt['impact_L'])
                )

fulladapt['extra_term'] = (
        (fulladapt['clip'] - histclim['clip']) 
        * ((histclim['highriskimpacts'] - histclim['high_base']) 
        - (histclim['lowriskimpacts'] - histclim['low_base']))
        )

# this is the impact calculated from the rearranged equation
fulladapt['impact'] = fulladapt['weighted_term'] + fulladapt['extra_term']

# this is the impact as calculated in the manual_rebasing_MC and manual_rebasing_singles scripts
fulladapt['impact_script'] = fulladapt['rebased_new'] - histclim['rebased_new']

In [52]:
# check Bangkok
fulladapt.loc[(fulladapt.regions == 'THA.3.R3edeff05b7928bfc')& (fulladapt.year == 2099)]

,operation,region,year,regions,rebased,lowriskimpacts,highriskimpacts,clip,orderofoperations,high_base,low_base,rebased_high,rebased_low,rebased_new,impact_H,impact_L,weighted_term,extra_term,impact,impact_script
2413198,0,20109,2099,THA.3.R3edeff05b7928bfc,-20.617348,25.850882,113.046776,0.332272,clip,130.349365,29.065708,-17.302589,-3.214827,-7.895799,-19.436958,-3.596285,-8.859701,0.21894,-8.64076,-8.64076


In [55]:
# check Winnipeg
fulladapt.loc[(fulladapt.regions == 'CAN.3.50.1276')& (fulladapt.year == 2099)]

,operation,region,year,regions,rebased,lowriskimpacts,highriskimpacts,clip,orderofoperations,high_base,low_base,rebased_high,rebased_low,rebased_new,impact_H,impact_L,weighted_term,extra_term,impact,impact_script
58918,0,490,2099,CAN.3.50.1276,5.174986,15.483075,70.138718,0.10051,clip,43.593613,9.566248,26.545105,5.916827,7.990174,24.341244,5.428177,7.329128,0.092734,7.421862,7.421862
